# CHỦ ĐỀ NGHIÊN CỨU
# SỬ DỤNG CÁC MẠNG RBF & KMEANS XỬ LÝ DATASET, PHÁT HIỆN BỆNH CÂY LÚA NƯỚC


In [ ]:
# 1. IMPORT THƯ VIỆN
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import sys

from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix
from scipy.spatial.distance import cdist

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Layer, Flatten,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [ ]:
#1.1 TẠO OUTPUT & LOG 
output_dir = 'output_rbf'
os.makedirs(output_dir,exist_ok=True)

log_file = open(os.path.join(output_dir,'log_output.txt'),'w')
sys.stdout = log_file

In [ ]:
# 2. CHUẨN BỊ DỮ LIỆU: CHIA DATASET THÀNH TRAIN, VAL, TEST VÀ THỐNG KÊ
original_data_dir = r'/kaggle/input/dataset/datasetriceleaf'  # Thư mục gốc chứa ảnh theo lớp
base_dir = 'rice_data_rbf_split'  # Thư mục mới để chứa dữ liệu chia sẵn
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

class_counts = {}  # Lưu số lượng ảnh theo lớp trước chia tách

# Liệt kê tên các lớp và đếm số lượng ảnh
class_names = []
for cls in os.listdir(original_data_dir):
    cls_path = os.path.join(original_data_dir, cls)
    if os.path.isdir(cls_path):
        imgs = os.listdir(cls_path)
        class_counts[cls] = len(imgs)
        class_names.append(cls)
print("Cac lop trong du lieu:")
for cls in sorted(class_names):
    print(f"- {cls}")

#Xóa thư mục chia cũ
if os.path.exists(base_dir):
    print("Dang xoa thu muc cu de chia lai...")
    shutil.rmtree(base_dir)

# Tạo lại thư mục chia dữ liệu
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)


#CHIA DỮ LIỆU
for cls in class_counts.keys():
    imgs = os.listdir(os.path.join(original_data_dir, cls))
    random.shuffle(imgs)
    n_total = len(imgs)
    n_train = int(n_total * 0.8)
    n_val = int(n_total * 0.1)

    for i, img in enumerate(imgs):
        src = os.path.join(original_data_dir, cls, img)
        if i < n_train:
            dst = os.path.join(train_dir, cls)
        elif i < n_train + n_val:
            dst = os.path.join(val_dir, cls)
        else:
            dst = os.path.join(test_dir, cls)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

#Nén thư mục chia 
shutil.make_archive('rice_data_rbf_split', 'zip', base_dir)
print("Da nen du lieu thanh file rice_data_rbf_split.zip, ban co the tai xuong file nay")

#THỐNG KÊ SỐ LƯỢNG ẢNH SAU KHI CHIA
for split in ['train', 'val', 'test']:
    print(f"\nThong ke so anh trong tap {split}:")
    split_path = os.path.join(base_dir, split)
    if os.path.exists(split_path):
        for cls in os.listdir(split_path):
            cls_path = os.path.join(split_path, cls)
            if os.path.isdir(cls_path):
                n = len(os.listdir(cls_path))
                print(f"- {cls}: {n} anh")
    else:
        print(f"Thu muc {split_path} khong ton tai.")

#KIỂM TRA THƯ MỤC ---
print("\nKiem tra ton tai thu muc:")
print("bas dir ton tai:", os.path.exists(base_dir))
print("Train dir ton tai:", os.path.exists(train_dir))
print("Val dir ton tai:", os.path.exists(val_dir))
print("Test dir ton tai:", os.path.exists(test_dir))

# VẼ BIỂU ĐỒ PHÂN BỐ SỐ LƯỢNG ẢNH BAN ĐẦU THEO LỚP
plt.figure(figsize=(10, 5))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title('So luong anh theo lop (truoc khi chia)')
plt.xlabel('Lop benh')
plt.ylabel('So luong anh')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir,'initial_class_distribution.png'))
plt.show()


In [ ]:
# 3. TIỀN XỬ LÝ DỮ LIỆU
img_size = (64, 64)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale = 1./255)

train_gen = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

#Lưu class_indices
class_indices = train_gen.class_indices
with open (os.path.join(output_dir,'class_indices.json'),'w') as f:
    json.dump(class_indices,f)
print("Đã lưu file class_indices.json.")

val_gen = val_test_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_gen = val_test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

num_classes = train_gen.num_classes
input_shape = (img_size[0], img_size[1], 3)

In [ ]:
# 4. CHUẨN BỊ TẬP DỮ LIỆU PHẲNG
def get_flattened_features(generator):
    features, labels = [], []
    for i in range(len(generator)):
        x, y = generator[i]  #Trả về 1 batch dữ liệu; x: batch ảnh,shape(batch_size, height, width, 3); y: batch nhãn, shape:(batch_size, num_classes)
        features.append(x)   #Lưu batch ảnh
        labels.append(y)     #Lưu batch nhãn
    #Gộp toàn bộ batch lại thành: 
    X = np.concatenate(features) #X:shape(total_shape, height, width, 3)
    y = np.concatenate(labels)   #Y:shape(total_sample, num_classes)
    return X.reshape(X.shape[0], -1), y #Làm phẳng ảnh từ 3D thành 1D- X.shape[0]:số ảnh;-1: tự suy ra số chiều còn lại= height*width*3

X_train_flat, y_train = get_flattened_features(train_gen)

In [ ]:
# 5. TÌM CENTER BẰNG KMEANS
n_centers = 300 #Lựa chọn 300 điểm trung tâm 
kmeans = KMeans(n_clusters=n_centers, random_state=42)
kmeans.fit(X_train_flat) #phân cụm Kmeans (X_train_flat có shape là (số ảnh,12288)- do ảnh resize (64,64,3))
centers = kmeans.cluster_centers_ #Trích xuất tọa độ 300 tâm (mỗi tâm là 1 vecto 12288 chiều)

In [ ]:
#5.1 Tính beta cho Gaussian từ kcach trung bình
#Tính khoảng cách giữa tất cả các cặp center
# dists = cdist(centers,centers, metric='euclidean')
# #Lấy trung binh kcach
# mean_dist = np.mean(dists[np.triu_indices_from(dists, k=1)])
# #Tính beta theo công thức 
# beta = 1.0/ (2*(mean_dist ** 2))
# print(f"Mean distance between centers:{mean_dist:.4f}, beta: {beta:.6f}")
dists = cdist(centers, centers, metric='euclidean')
pairwise_dists = dists[np.triu_indices_from(dists, k=1)]
median_dist = np.percentile(pairwise_dists, 50)  # Hoặc thử 25 hoặc 75

beta = 1.0 / (2 * (median_dist ** 2))
print(f"Median dist: {median_dist:.4f}, beta: {beta:.6f}")

In [ ]:
# 6. XÂY DỰNG LỚP RBF TÙY CHỈNH
class RBFLayer(Layer):
    def __init__(self, centers, beta, **kwargs): #Hàm khởi tạo lớp
        super(RBFLayer, self).__init__(**kwargs)     #gọi hàm khởi tạo của lớp Layer, tích hợp vào Keras
        self.centers_np = centers if isinstance(centers, np.ndarray) else np.array(centers)  
        self.centers = tf.constant(self.centers_np, dtype=tf.float32)
        self.beta = beta

    def call(self, inputs): #Hàm chính nhận đầu vào và trả về đầu ra lớp RBF
        inputs = tf.reshape(inputs, [tf.shape(inputs)[0], -1]) #chuyển thành ma trận 2D [batch_size, số_feature];tf.shape(inputs)[0]:lấy kích thước batch
        C = tf.expand_dims(self.centers, 0) #self.centers có shape là [num_centers, features]; tf.expand_dims: thêm chiều mới ở vị trí 0, tạo thành[1, num_centers, features]
        X = tf.expand_dims(inputs, 1) #inputs có shape[batch_size, features];thêm chiều mới ở vị trí 1, tạo thành [batch_size, 1, features]
        dist = tf.reduce_sum((X - C) ** 2, axis=-1) #X-C: mỗi điểm trong batch sẽ trừ với center tương ứng; tính tổng bình phương theo chiều features
        return tf.exp(-self.beta * dist) #Tính hàm RBF Gaussian: exp(-beta.|x-c|^2)
    def get_config(self):
        config = super().get_config()
        config.update({
            "centers": self.centers_np.tolist(),  # cần chuyển về list
            "beta": self.beta,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
# 7. XÂY DỰNG MÔ HÌNH RBF
inputs = Input(shape=(64,64,3)) #Tạo tầng đầu vào cho mô hình (64,64,3)
num_classes = y_train.shape[1] #số lớp phân loại

x = Flatten()(inputs) #CHuyển đầu vào thành dạng vecto 1 chiều
x = RBFLayer(centers, beta = beta)(x) #Thêm lớp ẩn RBF
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(num_classes, activation='softmax',kernel_regularizer=l2(0.001))(x) # Tầng phân loại đầu ra; num_classes: số lớp; softmax : chuẩn hóa đầu ra thành xsuat từng lớp

model = Model(inputs, outputs)
model.compile(optimizer= Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) #bộ tối ưu hóa adam; hàm mất mát cho bài toán phân loại, metrics theo dõi độ cxac 
model.summary() #Hiển thị kiến trúc mô hình 

#Callback EarlyStopping
callbacks = [
    EarlyStopping(patience = 5, restore_best_weights = True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose =1)
]
# 8. HUẤN LUYỆN
history = model.fit(train_gen, validation_data=val_gen, epochs=100, callbacks = callbacks)


In [ ]:
#8.1 Lưu model
model.save("/kaggle/working/rice_leaf_rbf_model.h5")

In [ ]:
# 9. ĐÁNH GIÁ
test_gen.reset()
pred = model.predict(test_gen)
y_pred = np.argmax(pred, axis=1)
y_true = test_gen.classes

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=test_gen.class_indices.keys(), yticklabels=test_gen.class_indices.keys(), cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig(os.path.join(output_dir,'confusion_matrix.png'))
plt.show()

In [ ]:
# 10. BIỂU ĐỒ ACCURACY VÀ LOSS
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Accuracy theo tung epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig(os.path.join(output_dir,'accuracy_plot.png'))
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss theo tung epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'loss_plot.png'))
plt.show()

In [ ]:
#11. ĐÓNG FILE LOG VÀ KHÔI PHỤC STDOUT
sys.stdout = sys.__stdout__
log_file.close()

In [ ]:
#12. ZIP FOLDER OUTPUT
shutil.make_archive("output_rbf",'zip',output_dir)